In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import re
import numpy as np
import tabulate
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
print("Fetching main faculty page...")
data = requests.get('https://www.iiserkol.ac.in/web/en/people/faculty/dbs/')
soup = BeautifulSoup(data.content, "html.parser")

# Collecting faculty profile URLs
urls = [link['href'] for link in soup.find_all('a')]
urls = [url for url in urls if '/web/en/people/faculty/dbs/' in url]
link = 'https://www.iiserkol.ac.in'
faculty_websites = list(set([link + url for url in urls]))
print(f"Found {len(faculty_websites)} faculty profile URLs.")

Fetching main faculty page...
Found 27 faculty profile URLs.


In [ ]:
all_data = []
print("Scraping individual faculty pages...")

for faculty_url in faculty_websites:
    try:
        data = requests.get(faculty_url)
        data.raise_for_status()
        soup = BeautifulSoup(data.content, "html.parser")

        # Get faculty name
        name = soup.find_all('h3')
        faculty_name = [name.text for name in name]
        faculty_name = faculty_name[3].strip() if len(faculty_name) > 3 else None

        if not faculty_name:
            print(f"Skipping URL (could not find name): {faculty_url}")
            continue

        print(f"Processing: {faculty_name}")

        # Initialize profile dictionary
        profile_data = {
            "Name": faculty_name,
            "Positions": None,
            "Current position": None,
            "Academic Background": None,
            "PhD": None,
            "PhD Year": None,
            "Years of Experience": None,
            "Research Interest": None,
            "Awards and Honors": None,
            "Number of awards": None,
        }

        # Get main profile content
        content = soup.find_all('div', class_='col-md-12 innerdiv')

        for section in content:
            section_text = section.get_text(separator="\n", strip=True)

            if 'Positions:' in section_text:
                profile_data["Positions"] = re.sub(r'Positions:\s+', '', section_text)
                positions_text = re.sub(r'Positions:\s+', '', section_text)
                profile_data["Positions"] = positions_text

                # 2. Now, try to find the specific "current" position
                current_position_found = False
                lines = positions_text.split('\n') # Split by newlines

                for line in lines:
                    if '(current)' in line.lower(): # Check for '(current)'
                        profile_data["Current position"] = line.strip()
                        current_position_found = True
                        break # Found it, stop looking

                # 3. Fallback: If no '(current)' found, assume the first non-empty line is current
                if not current_position_found and lines:
                    for first_line in lines:
                        first_line = first_line.strip()
                        if first_line: # Make sure it's not just an empty line
                            profile_data["Current position"] = first_line
                            break # Got the first one, stop


            elif 'Academic Background:' in section_text:
                profile_data["Academic Background"] = re.sub(r'Academic Background:\s+', '', section_text)

                # Try to extract PhD info from this section
                c = re.search(r"\d{4}", section_text) # Look for a 4-digit year
                if c:
                    x = c.start()
                    profile_data["PhD Year"] = section_text[x:x+4]
                    if 'PhD' in section_text or 'phd' in section_text or 'Ph.D' in section_text:
                        phd_text = section_text[21:x-1].strip(" ,")
                        profile_data["PhD"] = phd_text



            elif 'Research Interest:' in section_text:
                profile_data["Research Interest"] = re.sub(r'Research Interest:\s+', '', section_text)


            elif 'Awards and Honors:' in section_text:
                profile_data["Awards and Honors"] = re.sub(r'Awards and Honors:\s+', '', section_text)
                try:
                    award_count = (section_text.count('20') + section_text.count('19') -
                                   section_text.count('2020') - section_text.count('2019'))
                    profile_data["Number of awards"] = award_count
                except Exception:
                    profile_data["Number of awards"] = None # No awards found

        if profile_data["PhD Year"]:
            try:
                current_year = 2025 # Based on your system's date
                phd_year_int = int(profile_data["PhD Year"])

                # Basic validation
                if 1900 < phd_year_int <= current_year:
                    experience = current_year - phd_year_int
                    profile_data["Years of Experience"] = experience

            except (ValueError, TypeError):
                # This handles cases where PhD Year is None or not a valid number
                profile_data["Years of Experience"] = None


        all_data.append(profile_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {faculty_url}: {e}")
        continue

print("Scraping complete.")

In [ ]:
print("Connecting to database and saving data...")
conn = sqlite3.connect("faculty_data.db")
cursor = conn.cursor()

# Drop the table if it already exists
cursor.execute("DROP TABLE IF EXISTS Faculty")

# Create the new table
cursor.execute('''
CREATE TABLE Faculty (
    Name TEXT,
    Positions TEXT,
    Current_position TEXT,
    Academic_Background TEXT,
    PhD TEXT,
    PhD_Year INTEGER,
    Years_of_Experience INTEGER,
    Research_Interest TEXT,
    Awards_and_Honors TEXT,
    Number_of_awards INTEGER
)
''')

# Insert data into the table
for profile in all_data:
    cursor.execute('''
    INSERT INTO Faculty (
        Name, Positions, Current_position, Academic_Background, PhD, PhD_Year, Years_of_Experience, Research_Interest,
        Awards_and_Honors, Number_of_awards
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        profile["Name"],
        profile["Positions"],
        profile["Current position"],
        profile["Academic Background"],
        profile["PhD"],
        profile["PhD Year"],
        profile["Years of Experience"],
        profile["Research Interest"],
        profile["Awards and Honors"],
        profile["Number of awards"]
    ))

# Commit changes
conn.commit()
print("Data saved to faculty_data.db")

Connecting to database and saving data...
Data saved to faculty_data.db


In [ ]:
print("Loading data into pandas DataFrame...")
# Read data from SQL back into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM Faculty", conn)

# Close the database connection
# conn.close() # Removed this line to keep the connection open

# Define list of Indian locations
Indian_cities = [
    "Adilabad", "Adoni", "Agartala", "Agra", "Ahmedabad", "Ahmednagar", "Aizawl", "Ajmer", "Akola",
"Aligarh", "Allahabad", "Alwar", "Amravati", "Ambattur", "Amritsar", "Anand", "Anantapur",
"Andaman and Nicobar Islands", "Andhra Pradesh", "Arrah", "Arunachal Pradesh", "Asansol",
"Aurangabad", "Aurangabad (Bihar)", "Avadi", "Bally", "Balasore", "Bareilly", "Baramati",
"Baramulla", "Barasat", "Bardhaman", "Baripada", "Barmer", "Bathinda", "Begusarai", "Belgaum",
"Bellary", "Bengaluru", "Berhampur", "Bhagalpur", "Bhavnagar", "Bhilai", "Bhilwara", "Bhiwani",
"Bhopal", "Bhubaneswar", "Bhuj", "Bihar", "Bihar Sharif", "Bijapur", "Bilaspur", "Bilaspur (HP)",
"BITS Goa", "BITS Hyderabad", "BITS Pilani", "Bokaro", "Bombay", "Brahmapur", "Burdwan",
"Calcutta", "Chandigarh", "Chandrapur", "Chapra", "Chennai", "Chhattisgarh", "Chittoor", "Coimbatore",
"Cuddalore", "Cuttack", "Darbhanga", "Darjeeling", "Davanagere", "Dadra and Nagar Haveli and Daman and Diu",
"Dehradun", "Deoghar", "Delhi", "Dewas", "Dhanbad", "Dharwad", "Dhule", "Dibrugarh", "Dindigul",
"Diu", "Dumka", "Durg", "Durgapur", "Eluru", "Erode", "Faridabad", "Farrukhabad", "Firozabad",
"Gandhinagar", "Gaya", "Ghaziabad", "Giridih", "Goa", "Gopalganj", "Gorakhpur", "Gujarat",
"Gulbarga", "Guntur", "Guwahati", "Gwalior", "Hamirpur", "Haldia", "Haldwani", "Hapur", "Haridwar",
"Haryana", "Hazaribagh", "Himachal Pradesh", "Hindupur", "Hoshangabad", "Howrah", "Hubli-Dharwad",
"Hosur", "Hyderabad", "Ichalkaranji", "IIIT Allahabad", "IIIT Hyderabad", "IISc", "IISER Berhampur",
"IISER Bhopal", "IISER Kolkata", "IISER Mohali", "IISER Pune", "IISER Tirupati", "Imphal",
"Indore", "India", "Indian", "Jabalpur", "Jagdalpur", "Jadavpur", "Jaipur", "Jaisalmer", "Jalgaon",
"Jalna", "Jalpaiguri", "Jammu and Kashmir", "Jamnagar", "Jamui", "Jamshedpur", "Jhansi", "Jharkhand",
"Jhunjhunu", "JNU", "Jodhpur", "Junagadh", "Kadapa", "Kakinada", "Kalyan-Dombivli", "Kalyani",
"Kanchipuram", "Kanpur", "Kanyakumari", "Karaikal", "Karimnagar", "Karur", "Karnal", "Karnataka",
"Kathua", "Khammam", "Kharagpur", "Kochi", "Kolhapur", "Kolkata", "Kollam", "Korba", "Kota",
"Kothagudem", "Kozhikode", "Kulti", "Kumbakonam", "Kurnool", "Ladakh", "Lakshadweep", "Latur",
"Loni", "Lovely Professional University", "Lucknow", "Ludhiana", "Machilipatnam", "Madurai", "Madras",
"Mahbubnagar", "Maheshtala", "Malda", "Malegaon", "Mangalore", "Manipal", "Manipur", "Mandi",
"Mathura", "Mau", "Meerut", "Meghalaya", "Moradabad", "Morena", "Mumbai", "Muzaffarnagar",
"Muzaffarpur", "Mysuru", "Nagaland", "Nagpur", "Nainital", "Nanded", "Nashik", "Navi Mumbai",
"Nellore", "New Delhi", "NIT Calicut", "NIT Surathkal", "NIT Trichy", "NIT Warangal", "Nizamabad",
"Noida", "Odisha", "Ongole", "Ozhukarai", "Palakkad", "Panihati", "Panipat", "Parbhani", "Patiala",
"Patna", "Puducherry", "Pune", "Purnia", "Punjab", "Rajasthan", "Raiganj", "Raipur", "Rajahmundry",
"Rajkot", "Ranchi", "Rampur", "Ratlam", "Ratnagiri", "Rewa", "Rishikesh", "Ropar", "Roorkee",
"Rourkela", "Sagar", "Salem", "Samastipur", "Sambalpur", "Sangli", "Satara", "Satna", "Saharanpur",
"Shahjahanpur", "Shimla", "Shillong", "Shimoga", "Sikar", "Silchar", "Siliguri", "Siwan", "Solan",
"Solapur", "Sonipat", "South Dumdum", "Srikakulam", "Srinagar", "SRM Chennai", "Surat", "Tamil Nadu",
"Tanjavur", "Telangana", "Thanjavur", "Thiruvananthapuram", "Thrissur", "Tiruchirappalli", "Tirunelveli",
"Tirupati", "Tiruppur", "Tripura", "Tumkur", "Udaipur", "Ujjain", "Ulhasnagar", "Una", "Uttar Pradesh",
"Uttarakhand", "Vadodara", "Varanasi", "Vasai-Virar", "Vellore", "Vijayawada", "Visakhapatnam",
"Vizianagaram", "VIT Vellore", "Warangal", "Wardha", "West Bengal"

]

# Function to classify PhD institution
def institution(phd_string):
    if phd_string is None:
        return None
    for institute in Indian_cities:
        if institute in str(phd_string):
            return "India"
    return "Abroad"

# Apply the function to create a new column
df['PhD_Institution'] = df['PhD'].apply(institution)

# Create filtered DataFrames for further analysis
df_filtered = df[df['PhD_Institution'].notna()]
df_yrAw = df.dropna(subset=["Number_of_awards", "PhD_Year"])

# Prepare data for potential plotting or curve fitting
years = df_yrAw["PhD_Year"]
Aws = df_yrAw["Number_of_awards"]

print("Analysis complete. DataFrame 'df' is ready.")
print("\nDataFrame Head:")

# Just this on the last line, with NO print()
df


Loading data into pandas DataFrame...
Analysis complete. DataFrame 'df' is ready.

DataFrame Head:


,Name,Positions,Current_position,Academic_Background,PhD,PhD_Year,Years_of_Experience,Research_Interest,Awards_and_Honors,Number_of_awards,PhD_Institution
0,Neelanjana Sengupta,"Professor, IISER Kolkata (current)\nAssistant Professor, IISER Kolkata (2016 - 2019)\nAssociate Professor, Indian Institute of Science Education and Research Kolkata (2019 - 2019)\nScientist, National Chemical Laboratory (NCL) Pune (2008 - 2016)","Professor, IISER Kolkata (current)","Ph.D (Chemical and Materials Physics), UCI (University of California, Irvine), 2008, Advisor: Douglas J. Tobias","Ph.D (Chemical and Materials Physics), UCI (University of California, Irvine)",2008,17,"The key goal of our research is to understand and predict the behaviour of biological systems at the molecular level using ‘bottoms up’ approaches. This involves harnessing the power of modern-day computers to investigate the structure, dynamics and thermodynamics of biological complexes at the atomistic level in their closest native milieu, as well as under chemically perturbing environmental conditions. Advanced, state-of-the-art molecular simulations, combined with tools based on the principles of equilibrium and non-equilibrium statistical mechanics, offers us a powerful ‘computational microscope’ to investigate a variety of biological problams from a fundamental as well as from an applied perspective. Extensive collaborations with biochemists, disease biologists and material chemists are carried out. We are increasingly developing analytical tools that harness the advantages of machine learning (ML) and artificial intelligence methods (AI).","Bronze Medal\nfrom Chemical research Society of India (2024)\nHumboldt Experienced Researcher Fellowship\nfrom Alexander von Humboldt Foundation, Germany (2021)",2.0,Abroad
1,Rupak Datta,"Professor, IISER Kolkata (current)\nHead, Department of Biological Sciences, IISER Kolkata (2021 - 2024)\nAssistant & Associate Professor, IISER Kolkata (2010 - 2021)\nPostdoctoral Fellow, Department of Biochemistry & Molecular Biology, Saint Louis University (2006 - 2010)","Professor, IISER Kolkata (current)","PhD (Biochemistry & Molecular Biology), Indian Institute of Chemical Biology, 2006\nM.Sc (Biochemistry), University of Calcutta, 2000","PhD (Biochemistry & Molecular Biology), Indian Institute of Chemical Biology",2006,19,"Lysosomes are cell's recycling and waste disposal system, containing a battery acid hydrolases that degrade a wide variety of macromolecules and cellular debris into reusable forms. In doing so, lysosomes play a vital role in maintaining cellular homeostasis. They also function as key cell signaling hubs for regulating processes such as nutrient sensing and energy metabolism. The overarching goal of our research is to understand the role of this fascinating organelle in human health and disease. The details about our research can be found here: https://bio.iiserkol.ac.in/?page_id=4480","Young Investigator Award\nfrom International Congress of Human Genetics, Montreal (2011)\nRamalingaswami Fellowship\nfrom Department of Biotechnology, Government of India (2011)\nTravel award\nfrom Protein Mis-folding and Mis-processing in Disease meeting, National Institutes of Health (2009)\nPostdoctoral Presentation Finalist Award\nfrom American Society of Human Genetics, San Diego (2007)",4.0,India
2,Rahul Das,"Associate Professor, IISER Kolkata (current)\nAssistant Professor, IISER Kolkata (2014 - 2021)\nPostdoctoral Fellow, University of California, Berkeley (2009 - 2014)\nResearch Associate, Wockhardt Research Centre (2000 - 2001)","Associate Professor, IISER Kolkata (current)","Ph.D. (Structural Biology), (McMaster University, Canada), 2008\nM.Tech. (Biotech and Biochem Engg), (IIT Khargpur), 2000","Ph.D. (Structural Biology), (McMaster University, Canada)",2008,17,"The main interest of our laboratory is to understand the mechanism of signal transduction across the plasma membrane. Our research is focused on studying the allosteric

from matplotlib import pyplot as plt
_df_17['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_18['PhD_Year'].plot(kind='hist', bins=20, title='PhD_Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_19['Years_of_Experience'].plot(kind='hist', bins=20, title='Years_of_Experience')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_36['PhD_Institution'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_36, x='index', y='PhD_Institution', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_37['Current_position'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_37, x='PhD_Year', y='Current_position', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_38['PhD_Institution'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_38, x='PhD_Year', y='PhD_Institution', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
print("\nPhD Institution Counts:")

# Just this on the last line, with NO print()
df['PhD_Institution'].value_counts()

In [ ]:
from IPython.display import display

# --- 1. Define the columns you want to show
tableheadings = [
    'Name',
    'Positions',
    'Current_position',
    'Academic_Background',
    'PhD',
    'PhD_Year',
    'Years_of_Experience',
    'PhD_Institution',
    'Research_Interest',
    'Awards_and_Honors',
    'Number_of_awards'
]
dfnew = df[tableheadings]

# --- 2. Define your styles as a list of CSS rules ---
styles = [
    # --- Style for the whole table: borders ---
    {'selector': 'table',
     'props': [('border-collapse', 'collapse'),
               ('border', '2px solid black')]},

    # --- Style for all cells (th=header, td=data): borders and padding ---
    {'selector': 'th, td',
     'props': [('border', '1px solid #999'),
               ('padding', '8px'),
               ('text-align', 'left')]},

    # --- Style for just the header row: background color and font ---
    {'selector': 'th',
     'props': [('background-color', '#f2f2f2'),
               ('color', 'black'),
               ('font-weight', 'bold')]}
]

# --- 3. Apply the styles and create the styled object ---
styleddf = (dfnew.style
    # Apply all the border and header styles defined above
    .set_table_styles(styles)

    # --- Set column widths ---
    # Set a default width for all long columns
    .set_properties(**{'min-width': '200px', 'max-width': '300px'})

    # Set a specific width for short columns
    .set_properties(subset=['PhD_Year', 'Number_of_awards', 'PhD_Institution'], **{'min-width': '100px', 'max-width': '100px'})

    # --- Add color to a column ---
    # Applies a color gradient (heatmap) to the 'Number_of_awards' column
    .background_gradient(subset=['Years_of_Experience'], cmap='Blues')

    # --- Set row height (optional) ---
    .set_properties(**{'height': '50px'})
)

# --- 4. Display the final, styled table ---
display(styleddf)

,Name,Positions,Current_position,Academic_Background,PhD,PhD_Year,Years_of_Experience,PhD_Institution,Research_Interest,Awards_and_Honors,Number_of_awards
0,Neelanjana Sengupta,"Professor, IISER Kolkata (current) Assistant Professor, IISER Kolkata (2016 - 2019) Associate Professor, Indian Institute of Science Education and Research Kolkata (2019 - 2019) Scientist, National Chemical Laboratory (NCL) Pune (2008 - 2016)","Professor, IISER Kolkata (current)","Ph.D (Chemical and Materials Physics), UCI (University of California, Irvine), 2008, Advisor: Douglas J. Tobias","Ph.D (Chemical and Materials Physics), UCI (University of California, Irvine)",2008,17,Abroad,"The key goal of our research is to understand and predict the behaviour of biological systems at the molecular level using ‘bottoms up’ approaches. This involves harnessing the power of modern-day computers to investigate the structure, dynamics and thermodynamics of biological complexes at the atomistic level in their closest native milieu, as well as under chemically perturbing environmental conditions. Advanced, state-of-the-art molecular simulations, combined with tools based on the principles of equilibrium and non-equilibrium statistical mechanics, offers us a powerful ‘computational microscope’ to investigate a variety of biological problams from a fundamental as well as from an applied perspective. Extensive collaborations with biochemists, disease biologists and material chemists are carried out. We are increasingly developing analytical tools that harness the advantages of machine learning (ML) and artificial intelligence methods (AI).","Bronze Medal from Chemical research Society of India (2024) Humboldt Experienced Researcher Fellowship from Alexander von Humboldt Foundation, Germany (2021)",2.000000
1,Rupak Datta,"Professor, IISER Kolkata (current) Head, Department of Biological Sciences, IISER Kolkata (2021 - 2024) Assistant & Associate Professor, IISER Kolkata (2010 - 2021) Postdoctoral Fellow, Department of Biochemistry & Molecular Biology, Saint Louis University (2006 - 2010)","Professor, IISER Kolkata (current)","PhD (Biochemistry & Molecular Biology), Indian Institute of Chemical Biology, 2006 M.Sc (Biochemistry), University of Calcutta, 2000","PhD (Biochemistry & Molecular Biology), Indian Institute of Chemical Biology",2006,19,India,"Lysosomes are cell's recycling and waste disposal system, containing a battery acid hydrolases that degrade a wide variety of macromolecules and cellular debris into reusable forms. In doing so, lysosomes play a vital role in maintaining cellular homeostasis. They also function as key cell signaling hubs for regulating processes such as nutrient sensing and energy metabolism. The overarching goal of our research is to understand the role of this fascinating organelle in human health and disease. The details about our research can be found here: https://bio.iiserkol.ac.in/?page_id=4480","Young Investigator Award from International Congress of Human Genetics, Montreal (2011) Ramalingaswami Fellowship from Department of Biotechnology, Government of India (2011) Travel award from Protein Mis-folding and Mis-processing in Disease meeting, National Institutes of Health (2009) Postdoctoral Presentation Finalist Award from American Society of Human Genetics, San Diego (2007)",4.000000
2,Rahul Das,"Associate Professor, IISER Kolkata (current) Assistant Professor, IISER Kolkata (2014 - 2021) Postdoctoral Fellow, University of California, Berkeley (2009 - 2014) Research Associate, Wockhardt Research Centre (2000 - 2001)","Associate Professor, IISER Kolkata (current)","Ph.D. (Structural Biology), (McMaster University, Canada), 2008 M.Tech. (Biotech and Biochem Engg), (IIT Khargpur), 2000","Ph.D. (Structural Biology), (McMaster University, Canada)",2008,17,Abroad,"The main interest of our laboratory is to understand the mechanism of signal transduction across the plasma membrane. Our research is focused on studying the allosteric reg